# Run **Expedited Internet Bypass Protocol** on FABRIC Nodes

## Input Required Information

| Variable | Use |
| --- | --- |
| SLICE_NAME    | Name of the slice you wish to work on. |
| NODE_TO_FAIL | Node which will lose access to an interface. |
| INTF_TO_FAIL | Interface that will be failed on the node. |

In [ ]:
SLICE_NAME = "eibp-small-abhay"

#change based on slice details
NODE_TO_FAIL = "a1"
INTF_TO_FAIL = "eth2"

## Access the Slice

The orchestrator class is initalized, which also means the slice and its nodes are now accessable as well.

In [ ]:
from FabUtils import FabOrchestrator

try:
    manager = FabOrchestrator(SLICE_NAME)

except Exception as e:
    print(f"Exception: {e}")

Slice name: eibp-small-abhay
Slice and nodes were acquired successfully.


## Delete the Log from a Prior Test if Necessary

In [ ]:
rmLogCmd = "rm EIBP_*.log"
manager.executeCommandsParallel(rmLogCmd, prefixList="a,d,c")

Starting command on node c1
Command to execute: rm EIBP_*.log
Starting command on node d1
Command to execute: rm EIBP_*.log
Starting command on node d2
Command to execute: rm EIBP_*.log
Starting command on node a1
Command to execute: rm EIBP_*.log
Starting command on node a2
Command to execute: rm EIBP_*.log
Waiting for result from node c1
stdout: 
stderr: 
Waiting for result from node d1
stdout: 
stderr: 
Waiting for result from node d2
stdout: 
stderr: 
Waiting for result from node a1
stdout: 
stderr: 
Waiting for result from node a2
stdout: 
stderr: 


## EIBP **Initial Convergence**

Delay by a bit to get everything working first. The spines are started first, then 5 seconds later the leaves are.

In [ ]:
with open('config.txt','r') as file:
    config_data = {}

    for line in file:
        parts = line.strip().split(',')
        if len(parts) == 2:
            node_name, command = parts
            config_data[node_name] = command


    for node_name, command in config_data.items():
        print(f"Node:{node_name}, Command:{command}")

        startCmd = f"tmux new-session -d -s MNLR 'cd ~/MNLR_Code; {command}'"
        print(startCmd)

        manager.executeCommandsParallel(startCmd, prefixList = node_name)

Node:c1, Command:sudo ./MNLR -T 1 -L 1.1 -N 1
tmux new-session -d -s MNLR 'cd ~/MNLR_Code; sudo ./MNLR -T 1 -L 1.1 -N 1'
Starting command on node c1
Command to execute: tmux new-session -d -s MNLR 'cd ~/MNLR_Code; sudo ./MNLR -T 1 -L 1.1 -N 1'
Waiting for result from node c1
stdout: 
stderr: 
Node:d1, Command:sudo ./MNLR -T 2 -N 1
tmux new-session -d -s MNLR 'cd ~/MNLR_Code; sudo ./MNLR -T 2 -N 1'
Starting command on node d1
Command to execute: tmux new-session -d -s MNLR 'cd ~/MNLR_Code; sudo ./MNLR -T 2 -N 1'
Waiting for result from node d1
stdout: 
stderr: 
Node:d2, Command:sudo ./MNLR -T 2 -N 1
tmux new-session -d -s MNLR 'cd ~/MNLR_Code; sudo ./MNLR -T 2 -N 1'
Starting command on node d2
Command to execute: tmux new-session -d -s MNLR 'cd ~/MNLR_Code; sudo ./MNLR -T 2 -N 1'
Waiting for result from node d2
stdout: 
stderr: 
Node:a1, Command:sudo ./MNLR -T 3 -N 0 192.168.8.1 24 eth3
tmux new-session -d -s MNLR 'cd ~/MNLR_Code; sudo ./MNLR -T 3 -N 0 192.168.8.1 24 eth3'
Starting comm

# EIBP **Reconvergence** Testing

## Take the Interface down

In [ ]:
# Take the specified interface down
intfName = f"{NODE_TO_FAIL}-{INTF_TO_FAIL}-p1"
intf = manager.slice.get_interface(intfName)

intf.ip_link_down()

## Stop EIBP on Each Node

In [ ]:
stopCmd = "tmux kill-session -t MNLR"
manager.executeCommandsParallel(stopCmd, prefixList="a,d,c")

Starting command on node c1
Command to execute: tmux kill-session -t MNLR
Starting command on node d1
Command to execute: tmux kill-session -t MNLR
Starting command on node d2
Command to execute: tmux kill-session -t MNLR
Starting command on node a1
Command to execute: tmux kill-session -t MNLR
Starting command on node a2
Command to execute: tmux kill-session -t MNLR
Waiting for result from node c1
stdout: 
stderr: 
Waiting for result from node d1
stdout: 
stderr: 
Waiting for result from node d2
stdout: 
stderr: 
Waiting for result from node a1
stdout: 
stderr: 
Waiting for result from node a2
stdout: 
stderr: 


## Collect Log Results

Now that the nodes have logged updates to their respective log files, they need to be downloaded to be analyzed.

In [ ]:
# View contents on each node

# compileCmd = "ls"
# manager.executeCommandsParallel(compileCmd, prefixList="a,d,c")

In [ ]:
import os

LOG_DIR_PATH = "../logs"
LOG_NAME = "EIBP_{name}.log"
logPath = os.path.join(LOG_DIR_PATH, LOG_NAME)

# If the logs directory does not already exist, create it
if not os.path.exists(LOG_DIR_PATH):
    os.makedirs(LOG_DIR_PATH)

manager.downloadFilesParallel(logPath, LOG_NAME, prefixList="a,d,c", addNodeName=True)

Starting download on node c1
File to download: EIBP_c1.log
Location of download: ../logs/EIBP_c1.log
Starting download on node d1
File to download: EIBP_d1.log
Location of download: ../logs/EIBP_d1.log
Starting download on node d2
File to download: EIBP_d2.log
Location of download: ../logs/EIBP_d2.log
Starting download on node a1
File to download: EIBP_a1.log
Location of download: ../logs/EIBP_a1.log
Starting download on node a2
File to download: EIBP_a2.log
Location of download: ../logs/EIBP_a2.log
Waiting for result from node c1
Output: None
Waiting for result from node d1
Output: None
Waiting for result from node d2
Output: None
Waiting for result from node a1
Output: None
Waiting for result from node a2
Output: None


## Bring the Interface Back Up

In [ ]:
intf.ip_link_up()